<a href="https://colab.research.google.com/github/smcdonou02/McDonut/blob/main/TournamentDES_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies / Imports

In [ ]:
import simpy
import numpy as np
import pandas as pd

User Config

In [ ]:
CONFIG_FILE = "/content/drive/MyDrive/2024-2025 /_SSY611_Team_Project/Python Simulation Script/Simulation Script/ADCC_2025_US_Open - Orlando_FL_Config_From_Raw.xlsx"

SCENARIOS = {
    "A": {"no_show_dq": 0.07, "submission": 0.38, "points": 0.55},
    "B": {"no_show_dq": 0.05, "submission": 0.55, "points": 0.40},
    "C": {"no_show_dq": 0.15, "submission": 0.35, "points": 0.50},
    "D": {"no_show_dq": 0.05, "submission": 0.25, "points": 0.70},
}

SELECTED_SCENARIO = "B"

NUM_MATS = 4
CHECKIN_STAFF = 10
ARRIVAL_START = -30.0
ARRIVAL_END = 0.0
BRACKET_START = 0.0
CHECKIN_MIN = 0.5
CHECKIN_MAX = 1.5
BUFFER_MIN = 1.0
BUFFER_MAX = 3.0
RANDOM_SEED = 42

In [ ]:
#import data file, clean up any bad data
def load_tournament_divisions(path):
  df = pd.read_excel(path)
  df["Num_Competitors"] = pd.to_numeric(df["Num_Competitors"], errors="coerce").fillna(0).astype(int)
  df["Match_Time_Min"] = pd.to_numeric(df["Match_Time_Min"], errors="coerce").fillna(5.0).astype(float)
  return df

def compute_rounds(n):
    rounds = []
    c = n
    while c > 1:
        m = c // 2
        b = c % 2
        rounds.append((m, b))
        c = m + b
    return rounds

In [ ]:
class DivisionDES:
    def __init__(self, env, row, checkin_res, mat_res, scen, rng):
        self.env = env
        self.row = row
        self.div_name = row["Division_Code"]
        self.n = int(row["Num_Competitors"])
        self.match_time = float(row["Match_Time_Min"])
        self.start_time = 0.0
        self.rounds = compute_rounds(self.n)

        self.checkin_res = checkin_res
        self.mat_res = mat_res
        self.scen = scen
        self.rng = rng

        self.ready_store = simpy.Store(env)
        self.results = []
        self.div_finish = None

    # ---------------------
    # Competitor process
    # ---------------------
    def competitor(self, cid):
        # arrival uniform between ARRIVAL_START and ARRIVAL_END
        t_arr = self.rng.uniform(ARRIVAL_START, ARRIVAL_END)
        yield self.env.timeout(max(0, t_arr))

        # check-in
        with self.checkin_res.request() as req:
            yield req
            svc = self.rng.uniform(CHECKIN_MIN, CHECKIN_MAX)
            yield self.env.timeout(svc)

        yield self.ready_store.put(cid)

    # ---------------------
    # Match process
    # ---------------------
    def match(self, round_num, cidA, cidB):
      # --- get a specific mat ID ---
      mat_id = yield self.mat_res.get()
      start = self.env.now

      # Scenario outcome
      outcome = self.rng.choice(
        ["dq", "sub", "pts"],
        p=[self.scen["no_show_dq"],
           self.scen["submission"],
           self.scen["points"]]
      )

      if outcome == "dq":
        duration = 0.1
      elif outcome == "sub":
        duration = self.rng.uniform(0.5, self.match_time - 0.1)
      else:
        duration = self.match_time

      # Run match time
      yield self.env.timeout(duration)
      end = self.env.now

      yield self.mat_res.put(mat_id)

      # Pick winner
      winner = cidA if self.rng.random() < 0.5 else cidB

      self.results.append(dict(
        Division=self.div_name,
        Round=round_num,
        Match=(cidA, cidB),
        Winner=winner,
        Start=start,
        End=end,
        Duration=duration,
        Mat=mat_id
      ))

      return winner

    # ---------------------
    # Division driver
    # ---------------------
    def run(self):
        # start competitor arrivals/check-in
        for cid in range(1, self.n+1):
            self.env.process(self.competitor(cid))

        # Wait until bracket start
        yield self.env.timeout(BRACKET_START)

        # Start bracket progression
        competitors = []
        for _ in range(self.n):
            cid = yield self.ready_store.get()
            competitors.append(cid)

        # run rounds
        for r, (num_matches, byes) in enumerate(self.rounds, start=1):
            winners = []

            # matches
            procs = []
            idx = 0
            for _ in range(num_matches):
                A = competitors[idx]; B = competitors[idx+1]
                idx += 2
                p = self.env.process(self.match(r, A, B))
                procs.append(p)

            # byes
            if byes == 1:
                winners.append(competitors[-1])

            # wait for matches to finish
            finished = yield simpy.events.AllOf(self.env, procs)
            for out in finished.events:
                winners.append(out.value)

            competitors = winners  # next round

        self.div_finish = self.env.now

In [ ]:
def run_tournament(df, scenario="A", num_mats=4, staff=10, seed=42):
    rng = np.random.default_rng(seed)
    env = simpy.Environment()

    checkin_res = simpy.Resource(env, capacity=staff)
    mat_res = simpy.FilterStore(env, capacity=num_mats)
    mat_res.items = list(range(num_mats))  # mat IDs 0..num_mats-1

    scen = SCENARIOS[scenario]

    divisions = []
    for _, row in df.iterrows():
        if row["Num_Competitors"] < 2:
            continue
        d = DivisionDES(env, row, checkin_res, mat_res, scen, rng)
        divisions.append(d)
        env.process(d.run())

    env.run()

    # -------------------------------
    # Collect results
    # -------------------------------
    all_matches = []
    summary = []

    for d in divisions:
        all_matches.extend(d.results)
        summary.append({
            "Division": d.div_name,
            "Competitors": d.n,
            "Finish": d.div_finish
        })

    matches_df = pd.DataFrame(all_matches)
    summary_df = pd.DataFrame(summary)

    return {
        "matches": matches_df,
        "summary": summary_df,
        "tournament_finish": summary_df["Finish"].max(),
    }


In [ ]:
def monte_carlo(df, N=100, mats_list=[4,5,6], staff_list=[5,10,15], scenario="B"):
    results = []

    for mats in mats_list:
        for staff in staff_list:
            durations = []
            utilizations = []

            for i in range(N):
                out = run_tournament(df, scenario=scenario, num_mats=mats, staff=staff, seed=1000+i)
                dur = out["tournament_finish"]
                durations.append(dur)

            results.append(dict(
                mats=mats,
                staff=staff,
                avg_duration=np.mean(durations),
                p95_duration=np.percentile(durations,95),
            ))
    return pd.DataFrame(results)


In [ ]:
df = load_tournament_divisions(CONFIG_FILE)
result = run_tournament(df, scenario="A", num_mats=14, staff=4)

print(result["summary"])
print(result["matches"].head())
print("Tournament Duration:", result["tournament_finish"])

                       Division  Competitors      Finish
0            Intermediate_550kg            5   11.752776
1                Advanced_280kg            4   12.965118
2            Intermediate_280kg            4   12.495079
3            Intermediate_400kg            5   15.716334
4            Intermediate_200kg            5   16.484822
..                          ...          ...         ...
169      Adult15_Beginner_650kg            3  289.630069
170      Adult15_Beginner_650kg           10  304.627555
171  Adult15_Intermediate_800kg            2  292.274997
172      Adult15_Beginner_500kg            3  297.449436
173      Adult15_Beginner_760kg            8  301.282369

[174 rows x 3 columns]
             Division  Round   Match  Winner     Start        End  Duration  \
0  Intermediate_550kg      1  (1, 4)       4  1.652776   6.652776       5.0   
1  Intermediate_550kg      1  (2, 3)       3  1.652776   6.652776       5.0   
2  Intermediate_550kg      2  (5, 4)       5  6.652776 

In [ ]:
df = load_tournament_divisions(CONFIG_FILE)

# Run Monte Carlo across mats + staff
mc = monte_carlo(df, N=100, mats_list=[4,5,6], staff_list=[5,10,15])
print(mc)

   mats  staff  avg_duration  p95_duration
0     4      5    864.147004    883.183721
1     4     10    862.246016    878.832610
2     4     15    862.484751    881.704569
3     5      5    693.700760    708.963817
4     5     10    691.622816    705.067455
5     5     15    690.714643    704.445435
6     6      5    578.127186    589.953853
7     6     10    576.722338    586.846425
8     6     15    576.274156    588.776508
